In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

2.12.0


In [3]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [4]:
# filepath = "merged_u1-50_w001_resampled.csv"
filepath = "D:/online dataset csv files preprocessing/merged_u1-50_w001_resampled.csv"

df_original = pd.read_csv(filepath)

In [5]:
df_original

,timestamp,LAx,LAy,LAz,GYx,GYy,GYz,name
0,924313410000000,1.242813,-0.160181,-0.900440,0.194931,-0.004242,0.163818,1
1,924313420000000,1.117246,-0.187921,-1.529893,0.236458,0.076378,0.133293,1
2,924313430000000,0.912846,-0.192566,-1.570929,0.226074,0.012253,0.108246,1
3,924313440000000,0.874858,-0.227208,-1.453511,0.139336,-0.050667,0.085030,1
4,924313450000000,0.834022,-0.282976,-1.292671,-0.013374,-0.109314,0.078308,1
...,...,...,...,...,...,...,...,...
2055043,1368327120000000,-9.502644,-7.178468,4.246279,-2.834417,-0.120428,-0.987857,50
2055044,1368327130000000,-9.443804,-7.698220,3.059675,-2.638940,0.561997,-0.947715,50
2055045,1368327140000000,-8.345459,-6.805815,1.853457,-2.483606,1.225222,-0.954696,50
2055046,1368327150000000,-7.580540,-5.629017,1.559257,-2.429500,1.769765,-1.003565,50


In [6]:
df_original = df_original.drop("timestamp", axis = 1)

In [7]:
df_original['name'] = df_original['name'] - 1

In [8]:
df_original

,LAx,LAy,LAz,GYx,GYy,GYz,name
0,1.242813,-0.160181,-0.900440,0.194931,-0.004242,0.163818,0
1,1.117246,-0.187921,-1.529893,0.236458,0.076378,0.133293,0
2,0.912846,-0.192566,-1.570929,0.226074,0.012253,0.108246,0
3,0.874858,-0.227208,-1.453511,0.139336,-0.050667,0.085030,0
4,0.834022,-0.282976,-1.292671,-0.013374,-0.109314,0.078308,0
...,...,...,...,...,...,...,...
2055043,-9.502644,-7.178468,4.246279,-2.834417,-0.120428,-0.987857,49
2055044,-9.443804,-7.698220,3.059675,-2.638940,0.561997,-0.947715,49
2055045,-8.345459,-6.805815,1.853457,-2.483606,1.225222,-0.954696,49
2055046,-7.580540,-5.629017,1.559257,-2.429500,1.769765,-1.003565,49


In [9]:
X = df_original[['LAx', 'LAy', 'LAz','GYx', 'GYy', 'GYz']].copy()
y = df_original['name'].copy()

In [10]:
scaler = StandardScaler()
X_transformed = X.copy()
X_transformed = scaler.fit_transform(X_transformed)

scaled_X = pd.DataFrame(data = X_transformed, columns = ['LAx', 'LAy', 'LAz','GYx', 'GYy', 'GYz'])
scaled_X['name'] = y.values

scaled_X

,LAx,LAy,LAz,GYx,GYy,GYz,name
0,0.359745,-0.067021,-0.237446,0.105221,0.035341,0.175545,0
1,0.322625,-0.073783,-0.418039,0.133911,0.092852,0.143484,0
2,0.262201,-0.074915,-0.429812,0.126737,0.047108,0.117177,0
3,0.250972,-0.083358,-0.396125,0.066811,0.002223,0.092793,0
4,0.238900,-0.096951,-0.349979,-0.038693,-0.039614,0.085733,0
...,...,...,...,...,...,...,...
2055043,-2.816791,-1.777656,1.239172,-1.987696,-0.047543,-1.034070,49
2055044,-2.799397,-1.904341,0.898730,-1.852644,0.439278,-0.991908,49
2055045,-2.474708,-1.686826,0.552660,-1.745327,0.912402,-0.999240,49
2055046,-2.248585,-1.399994,0.468253,-1.707947,1.300863,-1.050568,49


In [11]:
frame_size = 600
hop_size = 600

In [37]:
df_original_600 = pd.DataFrame(columns = ['LAx', 'LAy', 'LAz','GYx', 'GYy', 'GYz','name'] )
# train = pd.DataFrame(columns = ['LAx', 'LAy', 'LAz','GYx', 'GYy', 'GYz','name'] )
# test = pd.DataFrame(columns = ['LAx', 'LAy', 'LAz','GYx', 'GYy', 'GYz','name'] )

rem_total = 0
labels = []
for i in range (0,50):
    filtered_df = df_original[df_original['name'] == i].copy()
    length = len(filtered_df)
    rem = length % 600
    final_i = filtered_df[:-rem]
    
    rem_total = rem_total + rem
    
    
#     df_train = final_i[:-600]
#     df_test = final_i[-600:]
    length_labels = int(len(final_i)/600)
    for j in range (0,length_labels):
        labels.append(i)
    
    
    
#     train = pd.concat([train, df_train])
#     test = pd.concat([test, df_test])
    
    df_original_600 = pd.concat([df_original_600, final_i])
print(rem_total)

20448


In [28]:
df_original_600

,LAx,LAy,LAz,GYx,GYy,GYz,name
0,1.242813,-0.160181,-0.900440,0.194931,-0.004242,0.163818,0
1,1.117246,-0.187921,-1.529893,0.236458,0.076378,0.133293,0
2,0.912846,-0.192566,-1.570929,0.226074,0.012253,0.108246,0
3,0.874858,-0.227208,-1.453511,0.139336,-0.050667,0.085030,0
4,0.834022,-0.282976,-1.292671,-0.013374,-0.109314,0.078308,0
...,...,...,...,...,...,...,...
2054455,0.862985,-1.314091,5.727084,-0.759219,0.575959,0.883137,49
2054456,2.530116,2.539922,9.561483,-0.068068,0.406662,0.621338,49
2054457,1.971137,4.040339,8.924051,0.740020,0.952951,0.502655,49
2054458,2.461469,4.971972,11.797399,1.226968,1.507966,0.581195,49


In [39]:
labels_array = np.array(labels)
labels_array.shape

(3391,)

In [44]:
X_values = df_original_600.drop('name', axis =1).values

In [45]:
X_values.shape

(2034600, 6)

In [46]:
sequence_length = 600
X_values_reshaped = np.reshape(X_values, (-1, sequence_length, 6))
X_values_reshaped.shape

(3391, 600, 6)

In [47]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X_values_reshaped, labels_array, test_size = 0.3, random_state = 0, stratify = labels_array)

X_train, X_validation, y_train, y_validation = train_test_split(X_train_val, y_train_val, test_size = 0.1, random_state = 0, stratify = y_train_val)
X_train.shape, X_test.shape, X_validation.shape, y_train.shape, y_test.shape, y_validation.shape

((2135, 600, 6), (1018, 600, 6), (238, 600, 6), (2135,), (1018,), (238,))

In [23]:
# X_train_values = train.drop("name", axis =1).values
# y_train_values = train['name'].values

# X_test_values = test.drop("name", axis =1).values
# y_test_values = test['name'].values

# X_train_values.shape , X_test_values.shape, y_train_values.shape, y_test_values.shape

((2004600, 6), (30000, 6), (2004600,), (30000,))

In [18]:
# sequence_length = 600
# train_reshaped = np.reshape(X_train_values, (-1, sequence_length, 6))
# test_reshaped = np.reshape(X_test_values, (-1, sequence_length, 6))

In [48]:
train_reshaped.shape, test_reshaped.shape

((3341, 600, 6), (50, 600, 6))

In [49]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(train_reshaped.shape[1], train_reshaped.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='softmax'))

In [50]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 598, 32)           608       
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 299, 32)          0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 297, 64)           6208      
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 148, 64)          0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 146, 128)          24704     
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 73, 128)         

In [51]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',
    min_delta = 0,
    patience = 2,
    verbose = 0,
    restore_best_weights = True
)

In [52]:
history = model.fit(X_train, y_train, epochs = 30,callbacks = [early_stopping], 
                     verbose=1)

Epoch 1/30


ValueError: in user code:

    File "C:\Users\szizo\anaconda3\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\szizo\anaconda3\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\szizo\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\szizo\anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\szizo\anaconda3\lib\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "C:\Users\szizo\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "